# 패키지 로드

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize

import os
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from PIL import Image

from tqdm import tqdm
import time

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

# 학습 설정

In [2]:
'''

학습 설정

추후에 argparser로 설정하면 될 듯

'''

train_data_dir = '../input/data/train/'
submission_data_dir = '../input/data/eval/'

train_image_dir = '../input/data/train/images'
submission_image_dir = '../input/data/eval/images'

model_dir = '../model/'

test_size = 0.2
fold_num = 5
seed = 22

lr = 0.005
device = 'cuda' if torch.cuda.is_available() else 'cpu'
batch_size = 128
epochs = 10

image_size = (128, 128)
image_normal_mean = (0.5, 0.5, 0.5)
image_normal_std = (0.2, 0.2, 0.2)

# 데이터 로드 및 전처리

In [3]:
df = pd.read_csv(train_data_dir + 'train.csv')
submission = pd.read_csv(submission_data_dir + 'info.csv')

In [4]:
'''

이상치 처리

'''

df.loc[df[df['id'] == '006359'].index, 'gender'] = 'male'
df.loc[df[df['id'] == '006359'].index, :]

,id,gender,race,age,path
2399,006359,male,Asian,18,006359_female_Asian_18


In [5]:
'''

학습 데이터셋 구축

'''
def get_age_group(x):
    if x < 30: return 0
    elif x < 60: return 1
    else: return 2

def get_gender_group(x):
    if x == 'male': return 0
    else: return 1

def get_train_df(df):
    train_df = []

    for idx, line in enumerate(tqdm(df.iloc)):
        for file in list(os.listdir(os.path.join(train_image_dir, line['path']))):
            if file[0] == '.':
                continue
            if file.split('.')[0] == 'normal':
                mask = 2
            elif file.split('.')[0] == 'incorrect_mask':
                mask = 1
            else:
                mask = 0
                
            mask_state = file.split('.')[0]
            gender_group = get_gender_group(line['gender'])
            age_group = get_age_group(line['age'])
            
            data = {
                'id' : line['id'],
                'gender' : line['gender'],
                'gender_group' : gender_group,
                'age' : line['age'],
                'age_group' : age_group,
                'mask_state': mask_state,
                'mask' : mask,
                'path': os.path.join(train_image_dir, line['path'], file),
                'label': mask * 6 + gender_group * 3 + age_group
            }
            
            train_df.append(data)
            
    train_df = pd.DataFrame(train_df)
    
    return train_df

train_df = get_train_df(df = df)

2700it [00:00, 2772.54it/s]


In [6]:
'''

데이터셋 분리

'''

train_idx, test_idx = train_test_split(train_df['label'], train_size = 1 - test_size, random_state = seed, stratify = train_df['label'])
                                      
train_set, test_set = train_df.iloc[train_idx.index, :].reset_index(drop = True), train_df.iloc[test_idx.index, :].reset_index(drop = True)

# 모델 설정

In [7]:
'''

ResNet 모델 구축

나동빈님 코드 참고

https://github.com/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/ResNet18_MNIST_Train.ipynb

'''

# ResNet18을 위해 최대한 간단히 수정한 BasicBlock 클래스 정의
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()

        # 3x3 필터를 사용 (너비와 높이를 줄일 때는 stride 값 조절)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        # 3x3 필터를 사용 (패딩을 1만큼 주기 때문에 너비와 높이가 동일)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        self.shortcut = nn.Sequential() # identity인 경우
        if stride != 1: # stride가 1이 아니라면, Identity mapping이 아닌 경우
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x) # (핵심) skip connection
        out = F.relu(out)
        return out


# ResNet 클래스 정의
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        # 64개의 3x3 필터(filter)를 사용
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes # 다음 레이어를 위해 채널 수 변경
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 16)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

# 데이터셋 구축

In [8]:
'''

Sample_submission 코드 참고

증강 데이터 생성은 본 함수에 추가할 것

데이터 셋 구축

'''

class CustomDataset(Dataset):
    def __init__(self, df, transform, train = True):
        self.train = train
        self.df = df
        if self.train:
            self.img_paths = self.df['path'].tolist()
            self.genders = self.df['gender_group'].tolist()
            self.ages = self.df['age_group'].tolist()
            self.masks = self.df['mask'].tolist()
            self.labels = self.df['label'].tolist()
        else:
            self.img_paths = [os.path.join(submission_image_dir, img_id) for img_id in self.df.ImageID]
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])
        if self.transform:
            image = self.transform(image)
        
        if self.train:
            label = torch.tensor(self.labels[index])
            age = torch.tensor(self.ages[index])
            gender = torch.tensor(self.genders[index])
            mask = torch.tensor(self.masks[index])
            return image, label, age, gender, mask
        
        else: return image

    def __len__(self):
        return len(self.img_paths)

# 학습 함수

In [9]:
def get_f1_score(y_true, y_pred):
    return f1_score(y_true, y_pred, average='macro')

def model_train(model, optimizer, criterion, images, targets):
    model.train()
    
    images, targets = images.to(device), targets.to(device)
    optimizer.zero_grad()
    
    benign_outputs = model(images)
    loss = criterion(benign_outputs, targets)
    loss.backward()
    
    optimizer.step()
    
    train_loss = loss.item()
    _, predicted = benign_outputs.max(1)
    
    correct = predicted.eq(targets).sum().item()
    
    fl_socre = get_f1_score(y_true = targets.cpu().tolist(), y_pred = predicted.cpu().tolist())
    
    return train_loss, correct, fl_socre

def batch_train(model, optimizer, criterion, images, targets, train_loss, train_acc, train_fi_score):
    loss, acc, fi_score = model_train(model = model, optimizer = optimizer, criterion = criterion, images = images, targets = targets)
    train_loss += loss
    train_acc += acc
    train_fi_score += fi_score

    return train_loss, train_acc, train_fi_score


def model_eval(model, criterion, images, targets):
    model.eval()
    with torch.no_grad():
        images, targets = images.to(device), targets.to(device)
        benign_outputs = model(images)
        loss = criterion(benign_outputs, targets)
        
        val_loss = loss.item()
        _, predicted = benign_outputs.max(1)
        
        correct = predicted.eq(targets).sum().item()
        fl_socre = get_f1_score(y_true = targets.cpu().tolist(), y_pred = predicted.cpu().tolist())
   
    return val_loss, correct, fl_socre

def batch_eval(model, criterion, images, targets, val_loss, val_acc, val_fi_score):
    loss, acc, fi_score = model_eval(model = model, criterion = criterion, images = images, targets = targets)
    val_loss += loss
    val_acc += acc
    val_fi_score += fi_score

    return val_loss, val_acc, val_fi_score

def ensemble(model, images):
    model.eval()
    all_predictions = []
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.softmax(dim = 1)
        all_predictions.extend(pred.cpu().numpy())
    
    return all_predictions

def get_loss(loss, acc, fi_score, data_loder, total):
    loss /= len(data_loder)
    acc /= total
    fi_score /= len(data_loder)

    return loss, acc, fi_score

# 학습

In [10]:
'''

데이터 전처리 함수

데이터 증강 등의 함수를 수정해서 사용하면 될 듯 함

'''

transform = transforms.Compose([
    Resize(image_size, Image.BILINEAR),
    ToTensor(),
    Normalize(mean=image_normal_mean, std=image_normal_std),
])

In [ ]:
'''

학습 시작

'''
total_start_time = time.time()

skf = StratifiedKFold(n_splits = fold_num, random_state = seed, shuffle = True)

criterion = nn.CrossEntropyLoss()

for now_fold_num, (trn_idx, val_idx) in enumerate(skf.split(train_set, train_set['label'])):
    fold_start_time = time.time()
    
    # 데이터 정의
    trn_df = train_set.iloc[trn_idx, :]
    val_df = train_set.iloc[val_idx, :]
    
    trn_dataset = CustomDataset(df = trn_df, 
                                transform = transform, 
                                train = True)
    
    val_dataset = CustomDataset(df = val_df, 
                                transform = transform, 
                                train = True)
    
    train_loder = DataLoader(
    trn_dataset,
    batch_size = batch_size,
    shuffle=True,
    )
    
    val_loder = DataLoader(
    val_dataset,
    batch_size = batch_size,
    shuffle = False,
    )
    
    # 모델 정의
    label_model = ResNet(block = BasicBlock, 
                         num_blocks = [2, 2, 2, 2],
                         num_classes = 18).to(device)
    
    mask_model = ResNet(block = BasicBlock, 
                        num_blocks = [2, 2, 2, 2], 
                        num_classes = 3).to(device)
    
    gender_model = ResNet(block = BasicBlock, 
                          num_blocks = [2, 2, 2, 2], 
                          num_classes = 2).to(device)
    
    age_model = ResNet(block = BasicBlock, 
                       num_blocks = [2, 2, 2, 2], 
                       num_classes = 3).to(device)
    
    # optimizer 설정
    label_optimizer = optim.Adam(label_model.parameters(), lr=lr)
    mask_optimizer = optim.Adam(mask_model.parameters(), lr=lr)
    gender_optimizer = optim.Adam(gender_model.parameters(), lr=lr)
    age_optimizer = optim.Adam(age_model.parameters(), lr=lr)
    
    # scheduler 설정
    label_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(label_optimizer, 'min', factor = 0.1, eps = 1e-09, patience = 5)
    mask_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(mask_optimizer, 'min', factor = 0.1, eps = 1e-09, patience = 5)
    gender_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(gender_optimizer, 'min', factor = 0.1, eps = 1e-09, patience = 5)
    age_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(age_optimizer, 'min', factor = 0.1, eps = 1e-09, patience = 5)
    
    # besf_metric 설정
    label_besf_fi = 0
    mask_besf_fi = 0
    gender_besf_fi = 0
    age_besf_fi = 0
    
    for epoch in range(1, epochs + 1):
        epoch_start_time = time.time()
        
        # 메트릭 정의
        total = 0
        train_label_loss, train_label_acc, train_label_fi_score, val_label_loss, val_label_acc, val_label_fi_score = 0, 0, 0, 0, 0, 0
        train_mask_loss, train_mask_acc, train_mask_fi_score, val_mask_loss, val_mask_acc, val_mask_fi_score = 0, 0, 0, 0, 0, 0
        train_gender_loss, train_gender_acc, train_gender_fi_score, val_gender_loss, val_gender_acc, val_gender_fi_score = 0, 0, 0, 0, 0, 0
        train_age_loss, train_age_acc, train_age_fi_score, val_age_loss, val_age_acc, val_age_fi_score = 0, 0, 0, 0, 0, 0
        
        # 학습
        for images, labels, ages, genders, masks in train_loder:
            total += labels.size(0)
            
            train_label_loss, train_label_acc, train_label_fi_score = batch_train(model = label_model, 
                                                                                   optimizer = label_optimizer, 
                                                                                   criterion = criterion, 
                                                                                   images = images, 
                                                                                   targets = labels, 
                                                                                   train_loss = train_label_loss, 
                                                                                   train_acc = train_label_acc, 
                                                                                   train_fi_score = train_label_fi_score)
            
            train_mask_loss, train_mask_acc, train_mask_fi_score = batch_train(model = mask_model, 
                                                                                   optimizer = mask_optimizer, 
                                                                                   criterion = criterion, 
                                                                                   images = images, 
                                                                                   targets = masks, 
                                                                                   train_loss = train_mask_loss, 
                                                                                   train_acc = train_mask_acc, 
                                                                                   train_fi_score = train_mask_fi_score)
            
            train_gender_loss, train_gender_acc, train_gender_fi_score = batch_train(model = gender_model, 
                                                                                   optimizer = gender_optimizer, 
                                                                                   criterion = criterion, 
                                                                                   images = images, 
                                                                                   targets = genders, 
                                                                                   train_loss = train_gender_loss, 
                                                                                   train_acc = train_gender_acc, 
                                                                                   train_fi_score = train_gender_fi_score)
            
            
            train_age_loss, train_age_acc, train_age_fi_score = batch_train(model = age_model, 
                                                                                   optimizer = age_optimizer, 
                                                                                   criterion = criterion, 
                                                                                   images = images, 
                                                                                   targets = ages, 
                                                                                   train_loss = train_age_loss, 
                                                                                   train_acc = train_age_acc, 
                                                                                   train_fi_score = train_age_fi_score)
            
        
        # total_loss 계산
        train_label_loss, train_label_acc, train_label_fi_score = get_loss(loss = train_label_loss, 
                                                                           acc = train_label_acc, 
                                                                           fi_score = train_label_fi_score, 
                                                                           data_loder = train_loder, 
                                                                           total = total)
        
        train_mask_loss, train_mask_acc, train_mask_fi_score = get_loss(loss = train_mask_loss, 
                                                                           acc = train_mask_acc, 
                                                                           fi_score = train_mask_fi_score, 
                                                                           data_loder = train_loder, 
                                                                           total = total)
        
        train_gender_loss, train_gender_acc, train_gender_fi_score = get_loss(loss = train_gender_loss, 
                                                                           acc = train_gender_acc, 
                                                                           fi_score = train_gender_fi_score, 
                                                                           data_loder = train_loder, 
                                                                           total = total)
        
        train_age_loss, train_age_acc, train_age_fi_score = get_loss(loss = train_age_loss, 
                                                                           acc = train_age_acc, 
                                                                           fi_score = train_age_fi_score, 
                                                                           data_loder = train_loder, 
                                                                           total = total)
        
        # 평가
        total = 0
        for images, labels, ages, genders, masks in val_loder:
            total += labels.size(0)
            
            val_label_loss, val_label_acc, val_label_fi_score = batch_eval(model = label_model, 
                                                                           criterion = criterion, 
                                                                           images = images, 
                                                                           targets = labels, 
                                                                           val_loss = val_label_loss, 
                                                                           val_acc = val_label_acc, 
                                                                           val_fi_score = val_label_fi_score)
            
            val_mask_loss, val_mask_acc, val_mask_fi_score = batch_eval(model = mask_model, 
                                                                           criterion = criterion, 
                                                                           images = images, 
                                                                           targets = masks, 
                                                                           val_loss = val_mask_loss, 
                                                                           val_acc = val_mask_acc, 
                                                                           val_fi_score = val_mask_fi_score)
            
            val_gender_loss, val_gender_acc, val_gender_fi_score = batch_eval(model = gender_model, 
                                                                           criterion = criterion, 
                                                                           images = images, 
                                                                           targets = genders, 
                                                                           val_loss = val_gender_loss, 
                                                                           val_acc = val_gender_acc, 
                                                                           val_fi_score = val_gender_fi_score)
            
            val_age_loss, val_age_acc, val_age_fi_score = batch_eval(model = age_model, 
                                                                           criterion = criterion, 
                                                                           images = images, 
                                                                           targets = ages, 
                                                                           val_loss = val_age_loss, 
                                                                           val_acc = val_age_acc, 
                                                                           val_fi_score = val_age_fi_score)
            
        
        # total_loss 계산
        val_label_loss, val_label_acc, val_label_fi_score = get_loss(loss = val_label_loss, 
                                                                           acc = val_label_acc, 
                                                                           fi_score = val_label_fi_score, 
                                                                           data_loder = val_loder, 
                                                                           total = total)
        
        val_mask_loss, val_mask_acc, val_mask_fi_score = get_loss(loss = val_mask_loss, 
                                                                           acc = val_mask_acc, 
                                                                           fi_score = val_mask_fi_score, 
                                                                           data_loder = val_loder, 
                                                                           total = total)
        
        val_gender_loss, val_gender_acc, val_gender_fi_score = get_loss(loss = val_gender_loss, 
                                                                           acc = val_gender_acc, 
                                                                           fi_score = val_gender_fi_score, 
                                                                           data_loder = val_loder,
                                                                           total = total)
        
        val_age_loss, val_age_acc, val_age_fi_score = get_loss(loss = val_age_loss, 
                                                                           acc = val_age_acc, 
                                                                           fi_score = val_age_fi_score, 
                                                                           data_loder = val_loder, 
                                                                           total = total)
    
        
        # 학습 결과 출력
        model_li = ['label', 'mask', 'gender', 'age']
        for idx, (train_loss, train_acc, train_f1, val_loss, val_acc, val_fi) in enumerate([
                [train_label_loss, train_label_acc, train_label_fi_score, val_label_loss, val_label_acc, val_label_fi_score],
                [train_mask_loss, train_mask_acc, train_mask_fi_score, val_mask_loss, val_mask_acc, val_mask_fi_score],
                [train_gender_loss, train_gender_acc, train_gender_fi_score, val_gender_loss, val_gender_acc, val_gender_fi_score],
                [train_age_loss, train_age_acc, train_age_fi_score, val_age_loss, val_age_acc, val_age_fi_score],
            ]):
            
            print(f'{now_fold_num + 1}fold, epoch: {epoch}, model: {model_li[idx]}, train_loss: {train_loss:.4f}, train_acc: {train_acc:.4f}, train_f1: {train_f1:.4f}, val_loss: {val_loss:.4f}, val_acc: {val_acc:.4f}, val_fi: {val_fi:.4f}')
        
        epoch_end_time = time.time()
        
        print(f'{now_fold_num + 1}fold, epoch: {epoch}, 학습시간: {epoch_end_time - epoch_start_time} \n')
        
        
        # 스케줄러
        label_scheduler.step(val_label_loss)
        mask_scheduler.step(val_mask_loss)
        gender_scheduler.step(val_gender_loss)
        age_scheduler.step(val_age_loss)
        
        
        # 모델 저장
        if label_besf_fi < val_label_fi_score:
            label_besf_fi = val_label_fi_score
            torch.save(label_model.state_dict(), model_dir + f'{now_fold_num + 1}fold_label_ResNet.pt')
        
        if mask_besf_fi < val_mask_fi_score:
            mask_besf_fi = val_mask_fi_score
            torch.save(mask_model.state_dict(), model_dir + f'{now_fold_num + 1}fold_mask_ResNet.pt')
            
        if gender_besf_fi < val_gender_fi_score:
            gender_besf_fi = val_gender_fi_score
            torch.save(gender_model.state_dict(), model_dir + f'{now_fold_num + 1}fold_gender_ResNet.pt')
            
        if age_besf_fi < val_age_fi_score:
            age_besf_fi = val_age_fi_score
            torch.save(age_model.state_dict(), model_dir + f'{now_fold_num + 1}fold_age_ResNet.pt')
    
    fold_end_time = time.time()
    
    print(f'{now_fold_num + 1}fold 훈련 시간: {fold_end_time - fold_start_time}')

total_end_time = time.time()
print(f'총 훈련 시간: {total_end_time - total_start_time}')

# 평가

In [ ]:
'''

평가 및 예측

'''

skf = StratifiedKFold(n_splits = fold_num, random_state = seed, shuffle = True)

# 앙상블 데이터 저장용
train_oof = np.zeros((train_set.shape[0], 4))
test_oof = np.zeros((test_set.shape[0], 4))
submission_oof = np.zeros((submission.shape[0], 4))

# test 데이터 정의
test_dataset = CustomDataset(df = test_set, 
                            transform = transform, 
                            train = True)

test_loder = DataLoader(test_dataset,
                        batch_size = batch_size,
                        shuffle = False,)

test_label_oof = np.zeros((test_set.shape[0], 18))
test_mask_oof = np.zeros((test_set.shape[0], 3))
test_gender_oof = np.zeros((test_set.shape[0], 2))
test_age_oof = np.zeros((test_set.shape[0], 3))

# submission 데이터 정의
submission_dataset = CustomDataset(df = submission, 
                            transform = transform, 
                            train = False)

submission_loder = DataLoader(submission_dataset,
                        batch_size = batch_size,
                        shuffle = False,)

submission_label_oof = np.zeros((submission.shape[0], 18))
submission_mask_oof = np.zeros((submission.shape[0], 3))
submission_gender_oof = np.zeros((submission.shape[0], 2))
submission_age_oof = np.zeros((submission.shape[0], 3))

# 예측
for now_fold_num, (trn_idx, val_idx) in tqdm(enumerate(skf.split(train_set, train_set['label']))):
    val_df = train_set.iloc[val_idx, :]
    
    val_dataset = CustomDataset(df = val_df, 
                                transform = transform, 
                                train = True)
    
    val_loder = DataLoader(val_dataset,
                            batch_size = batch_size,
                            shuffle = False,)
    
    
    # 모델 로드
    label_model = ResNet(block = BasicBlock, num_blocks = [2, 2, 2, 2], num_classes = 18).to(device)
    label_model.load_state_dict(torch.load(model_dir + f'{now_fold_num + 1}fold_label_ResNet.pt', map_location = device))
    
    mask_model = ResNet(block = BasicBlock, num_blocks = [2, 2, 2, 2], num_classes = 3).to(device)
    mask_model.load_state_dict(torch.load(model_dir + f'{now_fold_num + 1}fold_mask_ResNet.pt', map_location = device))
    
    gender_model = ResNet(block = BasicBlock, num_blocks = [2, 2, 2, 2], num_classes = 2).to(device)
    gender_model.load_state_dict(torch.load(model_dir + f'{now_fold_num + 1}fold_gender_ResNet.pt', map_location = device))
    
    age_model = ResNet(block = BasicBlock, num_blocks = [2, 2, 2, 2], num_classes = 3).to(device)
    age_model.load_state_dict(torch.load(model_dir + f'{now_fold_num + 1}fold_age_ResNet.pt', map_location = device))    
    
    # train_oof
    label_pred_li, mask_pred_li, gender_pred_li, age_pred_li = [], [], [], []
    with torch.no_grad():
        for images, _, _, _, _ in val_loder:
            images = images.to(device)
            
            label = label_model(images).argmax(dim=-1)
            mask = mask_model(images).argmax(dim=-1)
            gender = gender_model(images).argmax(dim=-1)
            age = age_model(images).argmax(dim=-1)
            
            label_pred_li.extend(label.cpu().numpy())
            mask_pred_li.extend(mask.cpu().numpy())
            gender_pred_li.extend(gender.cpu().numpy())
            age_pred_li.extend(age.cpu().numpy())
    
    train_oof[val_idx, 0] = label_pred_li
    train_oof[val_idx, 1] = mask_pred_li
    train_oof[val_idx, 2] = gender_pred_li
    train_oof[val_idx, 3] = age_pred_li
    
    # test_oof
    label_pred_li, mask_pred_li, gender_pred_li, age_pred_li = [], [], [], []
    with torch.no_grad():
        for images, _, _, _, _ in test_loder:
            images = images.to(device)
            
            label = label_model(images).softmax(1)
            mask = mask_model(images).softmax(1)
            gender = gender_model(images).softmax(1)
            age = age_model(images).softmax(1)
            
            label_pred_li.append(label.cpu().numpy())
            mask_pred_li.append(mask.cpu().numpy())
            gender_pred_li.append(gender.cpu().numpy())
            age_pred_li.append(age.cpu().numpy())

    test_label_oof += np.concatenate(label_pred_li) / fold_num
    test_mask_oof += np.concatenate(mask_pred_li)/ fold_num
    test_gender_oof +=np.concatenate(gender_pred_li) / fold_num
    test_age_oof += np.concatenate(age_pred_li) / fold_num
    
    # submission_oof
    label_pred_li, mask_pred_li, gender_pred_li, age_pred_li = [], [], [], []
    with torch.no_grad():
        for images in submission_loder:
            images = images.to(device)
            
            label = label_model(images).softmax(1)
            mask = mask_model(images).softmax(1)
            gender = gender_model(images).softmax(1)
            age = age_model(images).softmax(1)
            
            label_pred_li.append(label.cpu().numpy())
            mask_pred_li.append(mask.cpu().numpy())
            gender_pred_li.append(gender.cpu().numpy())
            age_pred_li.append(age.cpu().numpy())

    submission_label_oof += np.concatenate(label_pred_li) / fold_num
    submission_mask_oof += np.concatenate(mask_pred_li)/ fold_num
    submission_gender_oof +=np.concatenate(gender_pred_li) / fold_num
    submission_age_oof += np.concatenate(age_pred_li) / fold_num

test_oof[:, 0] = test_label_oof.argmax(1).tolist()
test_oof[:, 1] = test_mask_oof.argmax(1).tolist()
test_oof[:, 2] = test_gender_oof.argmax(1).tolist()
test_oof[:, 3] = test_age_oof.argmax(1).tolist()

submission_oof[:, 0] = submission_label_oof.argmax(1).tolist()
submission_oof[:, 1] = submission_mask_oof.argmax(1).tolist()
submission_oof[:, 2] = submission_gender_oof.argmax(1).tolist()
submission_oof[:, 3] = submission_age_oof.argmax(1).tolist()

In [ ]:
col_li = ['label', 'mask', 'gender_group', 'age_group']

ensemble_train_df = pd.DataFrame(train_oof, columns = col_li).astype(int)
ensemble_train_df['ensemble_label'] = ensemble_train_df['mask'] * 6 + ensemble_train_df['gender_group'] * 3 + ensemble_train_df['age_group']

ensemble_test_df = pd.DataFrame(test_oof, columns = col_li).astype(int)
ensemble_test_df['ensemble_label'] = ensemble_test_df['mask'] * 6 + ensemble_test_df['gender_group'] * 3 + ensemble_test_df['age_group']

ensemble_submission_df = pd.DataFrame(submission_oof, columns = col_li)
ensemble_submission_df['ensemble_label'] = ensemble_submission_df['mask'] * 6 + ensemble_submission_df['gender_group'] * 3 + ensemble_submission_df['age_group']

In [ ]:
'''

confusion_matrix and f1

'''

col_li = ['label', 'mask', 'gender_group', 'age_group', 'ensemble_label']

for col in col_li:
    if col != 'ensemble_label':
        train_y_true = train_set[col].values
        train_y_pred = ensemble_train_df[col].values
        
        test_y_true = test_set[col].values
        test_y_pred = ensemble_test_df[col].values
    
    else:
        train_y_true = train_set['label'].values
        train_y_pred = ensemble_train_df[col].values
        
        test_y_true = test_set['label'].values
        test_y_pred = ensemble_test_df[col].values        
    
    
    train_f1 = get_f1_score(y_true = train_y_true,  y_pred = train_y_pred)
    test_f1 = get_f1_score(y_true = test_y_true, y_pred = test_y_pred)
    
    train_confusion_matrix = pd.DataFrame((confusion_matrix(y_true = train_y_true, y_pred = train_y_pred)))
    test_confusion_matrix = pd.DataFrame((confusion_matrix(y_true = test_y_true, y_pred = test_y_pred)))
    
    print(f'{col} train confusion_matrix')
    display(train_confusion_matrix)
    
    print(f'{col} test confusion_matrix')
    display(test_confusion_matrix)
    print(f'train fi : {train_f1}, test_f1 fi : {test_f1}')
    print()

# 제출

In [ ]:
# 예측에 사용할 라벨 결정
submission['ans'] = ensemble_submission_df['ensemble_label'].astype(int).values

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(submission_data_dir, 'submission.csv'), index=False)
print('test inference is done!')

In [ ]:
submission.head()